<a href="https://colab.research.google.com/github/mlpbraga/sexism-detection-notebooks/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 22.6MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 
     |████████████████████████████████| 901kB 47.0MB/s 


In [2]:
import transformers
import torch
import math
import pandas as pd
from tqdm.notebook import trange, tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando: {device}")

Usando: cuda


In [4]:
class SentimentModel(torch.nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.pretrained_model = pretrained_model
        self.dense1           = torch.nn.Linear(in_features=768 * 512, out_features=512)
        self.dense2           = torch.nn.Linear(in_features=512, out_features=1)
        self.dropout          = torch.nn.Dropout(0.2)
        self.relu             = torch.nn.ReLU()
        self.sigmoid          = torch.nn.Sigmoid()

    def forward(self, X):
        y_hat = self.pretrained_model(X['input_ids'], X['attention_mask'])
        y_hat = y_hat[0].flatten(start_dim=1)
        y_hat = self.dense1(y_hat)
        y_hat = self.relu(y_hat)
        y_hat = self.dropout(y_hat)
        y_hat = self.dense2(y_hat)
        y_hat = self.sigmoid(y_hat)
        return y_hat

In [5]:
class DisasterDataset(torch.utils.data.Dataset):
  def __init__(self, X_input_ids, X_attention_mask, y):
    self.X_input_ids = X_input_ids
    self.X_attention_mask = X_attention_mask
    self.y = y

  def __getitem__(self, index):
    _x_ii = self.X_input_ids[index]
    _x_am = self.X_attention_mask[index]
    _y = self.y[index]
    return _x_ii, _x_am, _y

  def __len__(self):
    return len(self.X_input_ids)

In [ ]:
from sklearn.model_selection import train_test_split

labelad_data = pd.read_csv('https://raw.githubusercontent.com/mlpbraga/sexism-detection-notebooks/main/data/labeled-comments.csv')
dataset = labelad_data[['content','label']]

pretrained_model = transformers.AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device=device)
tokenizer = transformers.AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

epochs = 5
X, y = list(dataset['content']), list(dataset['label'])

metrics = []
for i in range(0, 10):
  print(f'-------------------------------- {i+1} -------------------------------------')
  X_train_original, X_test_original, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

  for param in pretrained_model.parameters():
    param.requires_grad = False

  X_train = tokenizer(X_train_original,
                    padding='max_length',
                    truncation=True,
                    max_length=512,
                    return_tensors='pt').to(device)

  y_train = torch.Tensor(y_train).to(device=device).view(-1, 1)

  dd_train = DisasterDataset(X_train['input_ids'], X_train['attention_mask'], y_train)
  dl_train = torch.utils.data.DataLoader(dd_train, batch_size=64, shuffle=True)

  sm = SentimentModel(pretrained_model).to(device)
  sm.train()

  criterion = torch.nn.BCELoss()
  optimizer = torch.optim.Adam(sm.parameters(), lr=0.0001)

  results = []
  losses_train_mean = []
  for epoch in range(epochs):
    losses_train = []
    losses_val = []
    for batch in tqdm(dl_train):
      X_ii_b, X_am_b, y_b = batch
      X_ii_b = X_ii_b.to(device)
      X_am_b = X_am_b.to(device)
      y_b = y_b.to(device)

      optimizer.zero_grad()
      output = sm.forward({
          'input_ids': X_ii_b,
          'attention_mask': X_am_b
      })

      loss_train = criterion(output, y_b)
      losses_train.append(loss_train.item())
      loss_train.backward()
      optimizer.step()

    geral_loss = sum(losses_train)/len(dl_train)
    losses_train_mean.append(geral_loss)
    print(f"Epoch: {epoch}, Loss: {round(geral_loss, 4)}")

  # test
  X_test = tokenizer(X_test_original,
                    padding='max_length',
                    truncation=True,
                    max_length=512,
                    return_tensors='pt').to('cpu')
  y_test = torch.Tensor(y_test).to(device='cpu').view(-1, 1)


  dd_test = DisasterDataset(X_test['input_ids'], X_test['attention_mask'], y_test)
  dl_test = torch.utils.data.DataLoader(dd_test, batch_size=64, shuffle=True)

  all_results = []
  for batch in tqdm(dl_test):
    X_ii_b, X_am_b, y_b = batch
    X_ii_b = X_ii_b.to(device)
    X_am_b = X_am_b.to(device)
    y_b = y_b.to(device)
    sm.eval()
    with torch.no_grad():
      results = sm.forward({
        'input_ids': X_ii_b,
        'attention_mask': X_am_b
      }).cpu()
      for element in results:
        all_results.append(element)

  y_pred = []
  for result in all_results:
    value = 1 if result > 0.5 else 0
    y_pred.append(value)

  y_test_ = []
  for result in y_test:
    value = int(result)
    y_test_.append(value)

  metrics.append((precision_score(y_test_, y_pred),
                  recall_score(y_test_, y_pred),
                  f1_score(y_test_, y_pred)))

  print(f'Metrics:', (precision_score(y_test_, y_pred),
                  recall_score(y_test_, y_pred),
                  f1_score(y_test_, y_pred)))
  print(classification_report(y_test_, y_pred))


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


-------------------------------- 1 -------------------------------------



Epoch: 0, Loss: 1.0954



Epoch: 1, Loss: 0.5749



Epoch: 2, Loss: 0.4981


...


Epoch: 4, Loss: 0.3688



Metrics: (0.5400516795865633, 0.5428571428571428, 0.5414507772020725)
              precision    recall  f1-score   support

           0       0.46      0.46      0.46       330
           1       0.54      0.54      0.54       385

    accuracy                           0.50       715
   macro avg       0.50      0.50      0.50       715
weighted avg       0.50      0.50      0.50       715

-------------------------------- 10 -------------------------------------



Epoch: 0, Loss: 1.0455



Epoch: 1, Loss: 0.5521



Epoch: 2, Loss: 0.4619


In [1]:
precision_0 = [0.48, 0.47,0.43,0.51,0.44,0.49,0.47,0.46,0.46,0.46]
precision_1 = [0.57, 0.55,0.51,0.58,0.52,0.57,0.56,0.54,0.54,0.53]
recall_0    = [0.71, 0.57,0.46,0.51,0.45,0.52,0.65,0.62,0.46,0.60]
recall_1    = [0.33,0.44,0.48,0.58,0.51,0.54,0.39,0.37,0.54,0.40]
f1_0        = [0.57,0.52,0.45,0.51,0.45,0.51,0.55,0.53,0.46,0.52]
f1_1        = [0.42,0.49,0.50,0.58,0.52,0.55,0.46,0.44,0.54,0.46]


In [2]:
def media(lista):
    return sum(lista)/10

In [4]:
media(precision_0),media(precision_1)

(0.46699999999999997, 0.547)

In [5]:
media(recall_0), media(recall_1)

(0.5549999999999999, 0.458)

In [6]:
media(f1_0), media(f1_1)

(0.507, 0.496)